In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./reddit_koreatravel_posts.csv")

In [3]:
df

,title,url,content,comments
0,South Korea (Seoul and Busan) Tips for Solo Tr...,https://www.reddit.com/r/koreatravel/comments/...,I went on a solo trip to South Korea from Febr...,100% Myeongdong is a trap. Not sure why it’s t...
1,Tips for experiencing the REAL Seoul (from a l...,https://www.reddit.com/r/koreatravel/comments/...,"Hi, my last post was bit promotional and I'm v...","Hmm, not sure why you’re not emphasizing the u..."
2,Some Insider Tips from a Seoul Local,https://www.reddit.com/r/koreatravel/comments/...,"Hey friends, huge thanks for the crazy support...",I've also been struggling with identity verifi...
3,A Local's Guide: Getting from Incheon Airport ...,https://www.reddit.com/r/koreatravel/comments/...,"Heyyyy, \nI'm back again today with some help...",Hands down will ride or die for the Limo Bus! ...
4,My third trip to korea. 2nd time solo.,https://www.reddit.com/r/koreatravel/comments/...,My second solo trip to seoul\n\nHello everyone...,I visit korea a few times already. Each time I...
...,...,...,...,...
95,Is June or July a Good Time to Travel to South...,https://www.reddit.com/r/koreatravel/comments/...,\n“Planning a trip to South Korea in June or J...,Not really. June/July are both humid + scatter...
96,3 weeks in Korea -any tips,https://www.reddit.com/r/koreatravel/comments/...,"Hi all, \n\nIm an Australian First time poster...","As a Korean, boring and Korea ain’t belong in ..."
97,I am a Korean student who loves to travel! Fee...,https://www.reddit.com/r/koreatravel/comments/...,"I'm a university student in Korea, and I'd lov...",It’s not my first time in Seoul and this time ...
98,Tax refund process at Incheon (step by step),https://www.reddit.com/r/koreatravel/comments/...,"Sorry if this has been posted before, but I al...",Thank you for positing this. \nThis is only in...


In [5]:
def combine_columns(row):
    return (
        f"제목: {row['title']}\n\n"
        f"본문 내용: {row['content']}\n\n"
        f"사용자 의견: {row['comments']}"
    )
    
df['full_text'] = df.apply(combine_columns, axis=1)

In [7]:
from langchain_community.document_loaders import DataFrameLoader
loader = DataFrameLoader(df, page_content_column="full_text")
documents = loader.load()

c:\Users\Admin\miniconda3\envs\rag-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700, # Adjust based on content density
    chunk_overlap=120,
    separators=["\n\n", "\n", ".", "!", "?", " "]
)
chunks = text_splitter.split_documents(documents)

In [ ]:
import os

# "sk-..." 형태의 실제 API 키를 입력하세요.
os.environ["OPENAI_API_KEY"] = "."
from langchain_openai import OpenAIEmbeddings
embedding_function = OpenAIEmbeddings(model="text-embedding-3-small")

In [21]:
from langchain_community.vectorstores import Chroma
vectordb = Chroma.from_documents(
    documents=chunks, 
    embedding=embedding_function, 
    persist_directory="./chroma_db_english_travel_tips"
)

In [6]:
vectordb.persist()
print("Chroma DB 적재 완료.")

NameError: name 'vectordb' is not defined

In [2]:
from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate

# 1. LLM 초기화
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)

# 2. Chroma DB를 Retriever로 변환
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# 3. 사용자 정의 프롬프트 템플릿 정의
system_prompt = """
You are a friendly and helpful Korea travel expert. 
Use the following context to answer the user's question accurately and concisely. 
If the information is not present in the context, state that you cannot answer based on the provided data.

Context: {context}
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

# 4. 문서 결합 체인과 검색 체인 구성
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# 5. 질문 실행 (테스트)
query = "What are the best tips for traveling alone in Seoul, according to the data?"

print(f"User Query: {query}\n")
print("--- LLM Response ---")

# 체인 실행
result = rag_chain.invoke({"input": query})
print(result['answer'])

ModuleNotFoundError: No module named 'langchain.chains'

In [3]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# 1. LLM 초기화
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)

# 2. Retriever 설정
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# 3. 프롬프트 템플릿
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a friendly and helpful Korea travel expert. 
Use the following context to answer the user's question accurately and concisely. 
If the information is not present in the context, state that you cannot answer based on the provided data.

Context: {context}"""),
    ("human", "{question}")
])

# 4. 간단한 RAG 함수
def ask_question(query):
    # 관련 문서 검색
    docs = retriever.get_relevant_documents(query)
    
    # 문서 내용을 하나의 문자열로 결합
    context = "\n\n".join([doc.page_content for doc in docs])
    
    # LLM에 질문
    messages = prompt.format_messages(context=context, question=query)
    response = llm.invoke(messages)
    
    return response.content

# 5. 질문 실행
query = "What are the best tips for traveling alone in Seoul, according to the data?"
print(f"User Query: {query}\n")
print("--- LLM Response ---")
print(ask_question(query))

ModuleNotFoundError: No module named 'langchain.prompts'

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
import os

os.environ["OPENAI_API_KEY"] = "."
embedding_function = OpenAIEmbeddings()

# Chroma DB 불러오기
vectordb = Chroma(
    persist_directory="./chroma_db_english_travel_tips",
    embedding_function=embedding_function
)

# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

# ✅ k값 안정적으로 설정 (예: 3~5 추천)
TOP_K = 5

def ask_question(query):
    # ✅ 유사도 점수 포함 검색
    results = vectordb.similarity_search_with_score(query, k=TOP_K)

    # 🔍 디버깅용 → 결과 점수 출력
    print("\n📌 Retrieved Documents with Similarity Scores:")
    context_list = []
    for i, (doc, score) in enumerate(results):
        print(f"Doc {i+1}: score={score:.4f}")
        # ✅ LLM에 넘길 문서 길이 줄이기
        context_list.append(doc.page_content[:400])  # 400자 제한

    context = "\n\n".join(context_list)

    # ✅ 컨텍스트 크기 확인 (토큰 안정)
    print(f"\n📏 Context length sent to LLM: {len(context)} characters")

    # 프롬프트 구성
    full_prompt = f"""
You are a friendly and helpful Korea travel expert.
Use ONLY the following context to answer the user's question accurately and concisely.
If the answer is not found in the context, say that the information is unavailable.

--- Context ---
{context}

--- Question ---
{query}
"""

    # LLM 질의
    response = llm.invoke(full_prompt)

    return response.content

# 테스트
query = "What are the best tips for traveling alone in Seoul, according to the data?"
print(f"\nUser Query: {query}")
print("\n--- LLM Answer ---")
print(ask_question(query))



User Query: What are the best tips for traveling alone in Seoul, according to the data?

--- LLM Answer ---

📌 Retrieved Documents with Similarity Scores:
Doc 1: score=1.8309
Doc 2: score=1.8328
Doc 3: score=1.8388
Doc 4: score=1.8452
Doc 5: score=1.8456

📏 Context length sent to LLM: 1837 characters
The information is unavailable.


In [10]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from dotenv import load_dotenv
import os

# ✅ .env 파일에서 환경변수 로드
load_dotenv()

embedding_function = OpenAIEmbeddings()

# ✅ 저장된 DB 불러오기
vectordb = Chroma(
    persist_directory="./chroma_db_english_travel_tips",
    embedding_function=embedding_function
)

# ✅ LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

# ✅ Retriever 설정
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# ✅ RAG 함수 개선
def ask_question(query, show_docs=False):
    docs = retriever.invoke(query)

    # 문서 없음 방지
    if not docs:
        return "No relevant data found in the knowledge base."

    # context 생성
    context = "\n\n".join([doc.page_content for doc in docs])

    # 시스템 + 사용자 프롬프트 분리 (더 안정적)
    messages = [
        {
            "role": "system",
            "content": (
                "You are a friendly and helpful Korea travel expert. "
                "Use ONLY the provided context to answer. "
                "If context is insufficient, say that you don't have enough information."
            ),
        },
        {
            "role": "user",
            "content": (
                f"--- Context ---\n{context}\n\n"
                f"--- User Question ---\n{query}"
            ),
        },
    ]

    # LLM 실행
    response = llm.invoke(messages).content

    # ✅ 원하면 관련 문서 내용 일부 같이 보여주기
    if show_docs:
        print("\n[📌 Retrieved Context Documents]")
        for i, doc in enumerate(docs):
            print(f"\n--- Doc #{i+1} ---")
            print(doc.page_content[:300], "...")

    return response


# ✅ 예시 실행
query = "What are the best tips for traveling alone in Seoul, according to the data?"
print(f"User Query: {query}\n")
print("--- LLM Response ---")
print(ask_question(query, show_docs=True))


User Query: What are the best tips for traveling alone in Seoul, according to the data?

--- LLM Response ---

[📌 Retrieved Context Documents]

--- Doc #1 ---
*National Museum of Korea*  
This major free museum should be on the top on your list. It has three floors full of amazing Korean historic artifacts and art, in a spectacular building with an amazing view. There are free English tours. It’s very big so be prepared to either spend a lot of time here, ...

--- Doc #2 ---
The **Museum of Contemporary Art** was a bit too highbrow for my taste (it’s also pretty far out of the centre). It’s nice to take a stroll in **Gamcheon village**, but it’s also very touristy. The **National Science Museum** is okay, little information in English here although the exhibits look spe ...

--- Doc #3 ---
Entrance is a bit expensive. You’ll pay at least 23.000 won (€14) for the ‘basic ticket’, 39.000 (€24) if you also want to see James Turrell and 46.000 (€28) for everything. Remember though that you’

In [11]:
print(vectordb._collection.count())


2513


In [13]:
ids = vectordb._collection.get(ids=None)["ids"]
print(ids[:10])

['4011ad68-2293-47ac-b972-3bbb553c5f9a', 'b481b4a1-06eb-4db8-85fb-7541817fccac', '5799136f-e485-4a39-81d2-f60a4e1b776f', '52a98d28-0ab9-43d5-80d7-ccc98eb39b8e', '17b65a0f-33f3-44b4-a020-06083d664668', '7fc38530-5f08-4074-ba22-c622bae0cee7', '66ba1b51-726d-46ed-b0e1-61897817edb6', 'ef49a9c1-c7cf-45ff-bb17-59626cdbc810', '49082918-99b4-4ca1-aba2-1d50e0622aee', '43f981e5-fe73-4e98-b4e9-1d021ac031f2']


In [14]:
data = vectordb._collection.get(include=["documents"])
for idx, doc in enumerate(data["documents"]):
    print(f"\n--- Document {idx+1} ---")
    print(doc)



--- Document 1 ---
제목: South Korea (Seoul and Busan) Tips for Solo Travelers

본문 내용: I went on a solo trip to South Korea from February 24 to March 2, 2025. I was in Seoul from February 24 to 27, then moved to Busan where I stayed until March 2, 2025. I know there are tons of tips and travel reports already online about South Korea so I won’t parrot back the same notes. I also did my research for weeks before my trip, yet I was still caught off guard with some aspects of South Korea that are not really talked about online.

--- Document 2 ---
For further context, I am not into K-Pop or Korean shows, and I went to the country purely to experience a foreign culture that is not Southeast Asian (almost finished with SEA trips at this point). These tips are not for beginners so please read other guides first as I will not discuss the basics here. I hope other fellow solo travelers can learn from my experience:

--- Document 3 ---
1. Most shops, restaurants, and tourist attractions don’t op

In [15]:
docs = vectordb.similarity_search("Seoul travel")
for d in docs:
    print(d.page_content[:300], "...")


. ||| I think we tried that once but i wouldn’t recommend it, it was a lot of hassle and the machines in the terminals are very user friendly. ||| Yeah you can only charge with cash. Unless you have something like Wowpass, they let you charge on the app with your credit card afaik. ||| Why you likel ...
. ||| Oh dang. You’re right about the payment, has to be a Korean card. Well as an alternative you can download the tmoney app and add an empty l t money card to your wallet. You will still have to top up at the transit center kiosk or a convenience store though.  But at least you’ll have the card o ...
**4. Payments**

* Korea is a card-friendly country. You can pay almost everything by card.
* Cash is useful only to top up your T-Money card and for street food stalls.
* This is important: bring at least **two different cards**, because sometimes (randomly) one card may not work with certain machi ...
# Days 9-10 - Seoul and home

We were staying in Insadong / Anguk area but I had to p

In [16]:
def inspect_chroma_db(vectordb, num_preview=5):
    print("\n📌 Chroma DB Info")
    print("-" * 40)
    print("Total Documents:", vectordb._collection.count())

    data = vectordb._collection.get(include=["documents", "metadatas"])
    docs = data["documents"]
    metas = data["metadatas"]

    print(f"\n📌 Preview ({num_preview} docs)")
    print("-" * 40)
    for i in range(min(num_preview, len(docs))):
        print(f"\n--- Document {i+1} ---")
        print("Metadata:", metas[i])
        print("Content:", docs[i][:300], "...")

inspect_chroma_db(vectordb)



📌 Chroma DB Info
----------------------------------------
Total Documents: 2513

📌 Preview (5 docs)
----------------------------------------

--- Document 1 ---
Metadata: {'title': 'South Korea (Seoul and Busan) Tips for Solo Travelers', 'comments': '100% Myeongdong is a trap. Not sure why it’s talked up so much on here. I had much more fun for way less money in Gangnam and Hongdae. ||| I’m not sure what a climate card is but essentially I used my T-Money card for transportation.\n\nI was also able to dine solo in many cases. The only time where it was not feasible and I had to order extra servings was in Busan when I went for charcoal grilled pork and when I tried to order from a machine myself for pork bone soup (the english translation portion was not work).\n\nI think it depends where you are dining but a lot of places accepted credit card. Even some of the smaller shops which surprised me.\n\nAs for Seoul, I love staying at the Moxy Insa-Dong. It is convenient with lots of good f